# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)
</PRE>


In [9]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# random seed a könnyebb reprodukálhatósághoz
seed = 77
np.random.seed(seed)

## Sentiment elemzés (tetszik/nem tetszik) szóbeágyazás alapján filmkritikákból

In [10]:
# Betöltjük az adatokat (Keras támogatott)
top_words=6000 # Limitáljuk a szavak számát, 6000 leggyakoribbat hagyjuk meg, többit kidobjuk
skip_top=0 # Abból is kihagyhatjuk mondjuk a 20 leggyakoribbat -> érdemes így is futtatni és összehasonlítani az eredményt

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words,skip_top=skip_top)

# Nincs validációs halmaz, lecsípünk a teszthalmazból validációhoz (25k/25k -> 25k/15k/10k)
X_val=X_test[:15000]
X_test=X_test[15000:]
y_val=y_test[:15000]
y_test=y_test[15000:]

# Adatok alakja, osztályok száma
print("Adataink: ")
print(X_train.shape)
print(y_train.shape)

print(y_val.shape)
print(y_test.shape)

print("Osztályok: ")
print(np.unique(y_train))

print("Szavak száma: ")
print(len(np.unique(np.hstack(X_train))))

print("Egy minta: ")
print(X_val[0])
print(y_val[0])

Adataink: 
(25000,)
(25000,)
(15000,)
(10000,)
Osztályok: 
[0 1]
Szavak száma: 
5998
Egy minta: 
[1, 323, 675, 2324, 314, 2553, 314, 2553, 1972, 2710, 314, 2, 1972, 10, 10, 1738, 3774, 9, 6, 2610, 3283, 3504, 37, 497, 8, 79, 34, 19, 27, 292, 5, 8, 30, 6, 52, 1246, 8, 27, 107, 2847, 103, 6, 5922, 4313, 39, 68, 452, 2, 21, 103, 35, 2, 19, 301, 2847, 984, 33, 6, 780, 2, 1333, 29, 1077, 6, 1189, 8, 68, 837, 707, 8, 193, 6, 2, 33, 2393, 5956, 2, 5, 27, 1366, 7, 254, 349, 37, 1374, 11, 4, 3560, 2, 879, 2, 2424, 50, 5, 8, 1374, 309, 83, 6, 2, 53, 2, 415, 159, 196, 240, 623, 4, 2, 7, 2393, 8, 483, 5, 47, 4657, 90, 17, 6, 2, 2, 17, 4, 2084, 2, 2, 3415, 11, 21, 54, 2, 2, 1091, 1701, 497, 8, 81, 6, 2, 11, 1392, 1951, 285, 505, 2, 2, 10, 10, 17, 670, 852, 4676, 280, 3431, 11, 4, 22, 6, 2, 787, 2, 57, 1963, 1521, 74, 1053, 676, 758, 4, 564, 7, 2, 69, 6, 52, 326, 5, 6, 676, 8, 97, 12, 83, 49, 432, 7, 22, 21, 209, 4, 1256, 12, 186, 8, 276, 12, 32, 295, 83, 49, 432, 7, 3984, 22, 10, 10, 17, 6, 2, 49, 

## Padding egyenlő hosszra

In [11]:
# Ennyi szót fogunk kritikánként figyelembe venni, ha ennél rövidebb kritika van, azt nullákkal kipaddingeli a pad_sequences().
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print("Első minta padding után")
print(X_train[0])

Első minta padding után
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

## Modell összeállítása, tanítás és tesztelés

In [12]:
embedding_dim=50

model = Sequential()
# Mi csináljuk az embeddinget is
model.add(Embedding(top_words, embedding_dim, input_length=max_words))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 50)           300000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 50)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               6250250   
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 6,550,501
Trainable params: 6,550,501
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Tanítás (csak 2 epoch, nincs túltanulás elleni védelmünk)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=128, verbose=1)

Train on 25000 samples, validate on 15000 samples
Epoch 1/2
25000/25000 [==============================] - 70s 3ms/step - loss: 0.5055 - acc: 0.7212 - val_loss: 0.2981 - val_acc: 0.8739
Epoch 2/2
25000/25000 [==============================] - 78s 3ms/step - loss: 0.1929 - acc: 0.9251 - val_loss: 0.3006 - val_acc: 0.8764


In [6]:
# Kiértékelés a tesztadatokon
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.86%
